##### Library

In [3]:
import os
import numpy as np
import pandas as pd

##### Load data

In [18]:
#current_dir = os.path.dirname(__file__)
current_dir = r'C:\Users\user\streamlit\src'
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_file_path = os.path.join(parent_dir, 'data', 'raw', 'raw_2024.xlsx')
df_original = pd.read_excel(data_file_path)

##### Feature Engineering

In [ ]:
df_gross = pd.DataFrame()

In [9]:
## Datetime
df_gross['date'] = df_original['결제일']
df_gross['year'] = df_original['결제일'].dt.year
df_gross['month'] = df_original['결제일'].dt.month
df_gross['day'] = df_original['결제일'].dt.day

weekday_map = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
df_gross['weekday'] = df_original['결제일'].dt.weekday.map(weekday_map)

df_gross['hour'] = df_original['출고일'].dt.hour

In [11]:
### Define Time
def convert_hour_to_text(hour):
    if hour == 0:
        return 'Other'
    elif 1 <= hour < 6:
        return 'Dawn'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Other'

df_gross['time'] = df_gross['hour'].apply(convert_hour_to_text)

In [14]:
## Channel
df_gross['channel'] = df_original['매출처']

def change_sales_channel(value):
    mapping = {
        '쿠팡윙': 'wing',
        '로켓그로스': 'growth',
        '스마트스토어': 'smartstore',
        '자사몰': 'cafe24'
    }
    return mapping.get(value, 'Other')

df_gross['channel'] = df_gross['channel'].apply(change_sales_channel)

In [16]:
## Product
df_gross['product'] = df_original['상품명']
df_original['옵션명'] = df_original['옵션명'].replace('기타', '1BOX')
df_gross['option'] = df_original['옵션명']

In [17]:
## Sales
df_gross['quantity'] = df_original['수량'].fillna(0)
df_gross['quantity'] = df_gross['quantity'].astype(int)
df_gross['sales'] = df_original['총\n결제가'].astype(int)

In [20]:
## Save
processed_dir_path = os.path.join(parent_dir, 'data', 'processed')
os.makedirs(processed_dir_path, exist_ok=True)
processed_file_path = os.path.join(processed_dir_path, 'gross.pkl')
df_gross.to_pickle(processed_file_path)